# DSI-ML-workshop: Deep Learning and Transfer Learning

Attribution: Kolhatkar, Varada (2024) DSCI572 

## Imports
<hr>

In [ ]:
import numpy as np
import pandas as pd
from collections import OrderedDict
import torch
from torch import nn, optim
from torchvision import datasets, transforms, utils, models
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from PIL import Image

plt.rcParams.update({'axes.grid': False})

<br><br>

## Getting Started with Kaggle Kernels
<hr>

We are going to run this notebook on the cloud using [Kaggle](https://www.kaggle.com). Kaggle offers 30 hours of free GPU usage per week which should be much more than enough for this lab. To get started, follow these steps:

1. Go to https://www.kaggle.com/kernels

2. Make an account if you don't have one, and verify your phone number (to get access to GPUs)
3. Select `+ New Notebook`
4. Go to `File -> Import Notebook`
5. Upload this notebook
6. On the right-hand side of your Kaggle notebook, make sure:
  
  - `Internet` is enabled.
  
  - In the `Accelerator` dropdown, choose `GPU` when you're ready to use it (you can turn it on/off as you need it).
7. In Kaggle Notebook, running the follow cell should print out `"Using device: cuda"` which means a GPU is available:

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device.type}")

Once you've done all your work on Kaggle, you can download the notebook from Kaggle. That way any work you did on Kaggle won't be lost. 

## Exercise 1: Transfer Learning
<hr>

In this exercise you're going to practice transfer learning. We're going to develop a model that can detect the following 6 cat breeds in this Kaggle [dataset](https://www.kaggle.com/solothok/cat-breed):

In order to use this dataset 

1. Click `+ Add data` at the top right of the notebook.

2. Search for 'cat-breed-mardhik'. Several datasets will appear. Look for and 'Add' the dataset with the size of 93 MB.

### Out-of-the-box Classification

Before the exercise, you will utilize the power of CNN model to classify some images of cats and dogs based on common cats and dogs image [dataset](https://www.kaggle.com/datasets/tongpython/cat-and-dog)! 

However, training a CNN model from scratch requires a lot of computation resources. So, you will leverage a pre-trained model (`DenseNet` for this case) to identify the animals!

In order to use the dataset.

1. Click `+ Add data` at the top right of the notebook.

2. Search for 'cat-and-dog'. Several datasets will appear. Look for and 'Add' the first dataset with the size of 228 MB.

3. Run the follow cell for preparation of the data, default labels and models.

In [ ]:
# Set up data
SAMPLE_DATA_DIR = "/kaggle/input/cat-and-dog/training_set/training_set"

def data_loader(DIR):
    IMAGE_SIZE = 200
    BATCH_SIZE = 32
    
    data_transforms = transforms.Compose([
        transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    
    dataset = datasets.ImageFolder(root=DIR, transform=data_transforms)
    loader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

    return loader

sample_data_loader = data_loader(SAMPLE_DATA_DIR)

def plot_samples(data_loader, model=None, classes=None):
    sample_batch = next(iter(data_loader))
    plt.figure(figsize=(20, 16)); 
    plt.axis("off"); 
    plt.title("Sample Images")
    plt.imshow(np.transpose(utils.make_grid(sample_batch[0], padding=1, normalize=True),(1, 2, 0)));
    actual_labels = sample_batch[1].numpy()
    print(f"Actual Labels: {actual_labels}")
    if model:
        model.to(device)
        _, y_hat_labels = torch.softmax(model(sample_batch[0].to(device)), dim=1).topk(1, dim=1)
        predicted_labels = y_hat_labels.squeeze().cpu().numpy()
        accuracy = np.mean(actual_labels == predicted_labels)
        if classes:
            predicted_labels = [classes[label] for label in predicted_labels]
        print(f"Predicted Labels: {predicted_labels}")
        print(f"Accuracy: {accuracy}")
        
    return

In [ ]:
# Download ImageNet labels
!wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt

In [ ]:
# Read the ImageNet categories
with open("imagenet_classes.txt", "r") as f:
    pretrained_categories = [s.strip() for s in f.readlines()]

In [ ]:
# Download model and freeze params
cnn_model = models.densenet121(pretrained=True)
for param in cnn_model.parameters():
    param.requires_grad = False

Then, you will obtain the predictions (`Predicted Labels`) from the model by running the follow cell. 

In [ ]:
# Time to predict
plot_samples(sample_data_loader, cnn_model, pretrained_categories)

**Food for Thought**: 

- How well can model identify cats and dogs?
- Anything you notice about the default labels?

_Type your answer here, replacing this text._

Now, we're going to develop a model that can detect the following 6 cat breeds in this Kaggle [dataset](https://www.kaggle.com/solothok/cat-breed):

0. American Short hair
1. Bengal
2. Maine Soon
3. Ragdoll
4. Scottish Fold
5. Sphinx

**Your tasks:**

In this exercise, you will build a CNN model to classify images of cats based on their breeds! 

You will leverage a pre-trained model (`DenseNet` for this case) to identify various cat breeds!

First, run the follow cells for preparation of the data.

In [ ]:
# Set up data
TRAIN_DIR = "/kaggle/input/cat-breed/cat-breed/TRAIN/"
VALID_DIR = "/kaggle/input/cat-breed/cat-breed/TEST/"

train_loader = data_loader(TRAIN_DIR)
valid_loader = data_loader(VALID_DIR)

Then, you will obtain the predictions (`Predicted Labels`) from the model by running the follow cell.

In [ ]:
# Time to predict
plot_samples(train_loader, cnn_model, pretrained_categories)

**Food for Thought**: 

- How is the performance of the model for this dataset?
- Do you think it does well in predicting the specific labels that we need?

_Type your answer here, replacing this text._

### Feature Extractor

**Your tasks:**

In this exercise, you will train a CNN model customized with your own layer(s) on top in order to build a CNN classifier that can identify specific cat breeds!

First, run the follow cell for preparation of the model training setup.

In [ ]:
# Set up trainer
def trainer(model, train_loader, valid_loader, epochs=10, verbose=True):
    """Simple training wrapper for PyTorch network."""

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.002)
    
    train_loss, valid_loss, train_accuracy, valid_accuracy = [], [], [], []
    for epoch in range(epochs):  # for each epoch
        train_batch_loss = 0
        train_batch_acc = 0
        valid_batch_loss = 0
        valid_batch_acc = 0
        
        # Training
        model.train()
        for X, y in train_loader:
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            y_hat = model(X)
            _, y_hat_labels = torch.softmax(y_hat, dim=1).topk(1, dim=1)
            loss = criterion(y_hat, y)
            loss.backward()
            optimizer.step()
            train_batch_loss += loss.item()
            train_batch_acc += (y_hat_labels.squeeze() == y).type(torch.float32).mean().item()
        train_loss.append(train_batch_loss / len(train_loader))
        train_accuracy.append(train_batch_acc / len(train_loader))
        
        # Validation
        model.eval()
        with torch.no_grad():
            for X, y in valid_loader:
                X, y = X.to(device), y.to(device)
                y_hat = model(X)
                _, y_hat_labels = torch.softmax(y_hat, dim=1).topk(1, dim=1)
                loss = criterion(y_hat, y)
                valid_batch_loss += loss.item()
                valid_batch_acc += (y_hat_labels.squeeze() == y).type(torch.float32).mean().item()
        valid_loss.append(valid_batch_loss / len(valid_loader))
        valid_accuracy.append(valid_batch_acc / len(valid_loader))
        
        # Print progress
        if verbose:
            print(f"Epoch {epoch + 1}:",
                  f"Train Accuracy: {train_accuracy[-1]:.2f}",
                  f"Valid Accuracy: {valid_accuracy[-1]:.2f}")
    
    results = {"train_loss": train_loss,
               "train_accuracy": train_accuracy,
               "valid_loss": valid_loss,
               "valid_accuracy": valid_accuracy}
    return results

> If you want to take a look at the images in training set, try this code:

In [ ]:
# Plot samples
plot_samples(train_loader)

Then, you will use a pre-trained model (`DenseNet` for this case), define the classification layer for our specific classes, and start training your model by running the follow cells.

In [ ]:
# Download model and freeze params
cnn_model = models.densenet121(pretrained=True)
for param in cnn_model.parameters():
    param.requires_grad = False

# Customize final classification layers
new_layers = nn.Sequential(
    nn.Linear(1024, 50),
    nn.ReLU(),
    nn.Linear(50, 6)
)
cnn_model.classifier = new_layers

In [ ]:
# Time to train
results = trainer(cnn_model, train_loader, valid_loader)

> If you want to take a look at the images in validation set and compare the actual and the predicted labels, try this code:

In [ ]:
# Plot samples
plot_samples(valid_loader, cnn_model)

**Food for Thought**: 

- How is the performance of the model? Compare the performance of out-of-the-box model and the feature extractor model.

_Type your answer here, replacing this text._

### Your Free Time (Optional)

**Your tasks**:

You will add any image dataset you like and train your own model with your own settings!

Feel free to share your thoughts with your teammates and workshop team.

In [ ]:
TRAIN_DIR = "{_TRAIN_FILE_PATH_}"
VALID_DIR = "{_VALID_FILE_PATH_}"
# Example
# TRAIN_DIR = "/kaggle/input/cat-breed/cat-breed/TRAIN/"
# VALID_DIR = "/kaggle/input/cat-breed/cat-breed/TEST/"

train_loader = data_loader(TRAIN_DIR)
valid_loader = data_loader(VALID_DIR)

In [ ]:
# Plot samples
plot_samples(train_loader)

In [ ]:
num_label_classes = "{_num_label_classes_}"
# Example
# num_label_classes = 6

# Download model and freeze params
cnn_model = models.densenet121(pretrained=True)
for param in cnn_model.parameters():
    param.requires_grad = False

# Customize final classification layers
new_layers = nn.Sequential(
    nn.Linear(1024, 50),
    nn.ReLU(),
    nn.Linear(50, num_label_classes)
)
cnn_model.classifier = new_layers

In [ ]:
# Time to train
results = trainer(cnn_model, train_loader, valid_loader)

In [ ]:
# Plot samples
plot_samples(valid_loader, cnn_model)

<!-- END QUESTION -->

<br><br>

<br><br>